In [1]:
"""
    使用XGB    max_depth=12      Score=
                          15?
"""
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('data/data74063/train.csv')
test1 = pd.read_csv('./test1.csv')

features = train.drop(['Unnamed: 0','label'],axis = 1)
labels = train['label']

FileNotFoundError: [Errno 2] No such file or directory: 'data/data74063/train.csv'

#### 构造新特征，特征比大于15的特征为关键特征

In [ ]:
#数据探索，找到导致1的关键特征值
def find_key_feature(train,selected):
    temp = pd.DataFrame(columns = [0,1])
    temp[0] = train[train['label']==0][selected].value_counts()/len(train[train['label']==0]) 
    temp[1] = train[train['label']==1][selected].value_counts()/len(train[train['label']==1]) 
    temp[2] = temp[1]/temp[0]
    #选出大于15倍的特征
    result = temp[temp[2]>15].sort_values(2,ascending = False).index
    return result

key_features = {}
selected_col = ['osv','apptype','carrier','dev_height','dev_ppi','dev_width',
                'media_id','ntt','version','location','fea1_hash','cus_type']
for selected in selected_col:
    key_features[selected] = find_key_feature(train,selected)

#构造新特征，新特征字段 = 原始特征字段+1
def f1(x,selected):
    #判断是否在关键特征值里，是1，否0
    if x in key_features[selected]:
        return 1
    else:
        return 0

for selected in selected_col:
    if len(key_features[selected]) > 0:
        features[selected + '1'] = features[selected].apply(f1,args = (selected,))
        test1[selected + '1'] = test1[selected].apply(f1,args = (selected,))
        print(selected+'1 created')

In [ ]:
train

#### osv转换为float类型

In [4]:
#对osv进行数据清洗
import re
def f2(x):
    x = str(x)
    x = x.replace('Android_','')
    # 4.2.3.2是需要找到的字符
    result = re.match('[\d\.]+',x)
    if result:
        x = result.group()    #返回匹配结果
        if '.' in x:
            #把4.4.3转化成4.43
            #找到第一个小数点的位置
            index1 = x.index('.')
            if index1 > 0:
                #去掉所有小数点
                x = x.replace('.','')
                #加上原来第一个小数点
                x = float(x[0:index1]+'.'+x[index1:])
        else: x = float(x)
    else:
        x = 0
    #如果版本号过大，7930
    if x>1000:
        x = x/10000
    elif x>11:
        x = x/10
    return x
    
features['osv'] = features['osv'].apply(f2)
test1['osv'] = test1['osv'].apply(f2)

#### 特征筛选

In [5]:
#类别特征
cate_features = ['apptype','carrier','ntt','version','location','cus_type']
remove_list = ['os','lan','sid']
col = features.columns.tolist()
for i in remove_list:
    col.remove(i)
features = features[col]

#### 时间

In [6]:
import time  
from datetime import datetime  

def get_date(features):
    #除以1000，转换为日期格式
    features['timestamp'] = features['timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
    
    #创建时间戳索引
    temp = pd.DatetimeIndex(features['timestamp'])
    features['year'] = temp.year
    features['month'] = temp.month
    features['day'] = temp.day
    features['week_day'] = temp.weekday
    features['hour'] = temp.hour
    features['minute'] = temp.minute

    #添加time_diff
    start_time = features['timestamp'].min()
    features['time_diff'] = features['timestamp'] - start_time
    
    #将time_diff转换为小时
    features['time_diff'] = features['time_diff'].dt.days * 24 +features['time_diff'].dt.seconds/3600
    
    #使用day，time_diff
    features.drop(['timestamp','year','month','minute','week_day'],axis = 1,inplace = True)
  
    return features

#对训练集提取时间多尺度
features = get_date(features)
#对测试集提取时间多尺度
test1 = get_date(test1)

#### LabelEncoder

In [7]:
#对lan进行特征编码LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#需要将训练集和测试集合并，然后统一做LabelEncoder
all_df = pd.concat([train,test1])
all_df['lan'] = all_df['lan'].astype('str')
all_df['lan'] = le.fit_transform(all_df['lan'])

#### 数据清洗

In [8]:
# 对于数值过大的异常值，设置为0
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

#对数据清洗，将V3=>3,V1=>1,V6=>6,V2=>2
#针对version，非数值类型 设置0
features['version'] = features['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
features['lan'] = all_df[all_df['label'].notnull()]['lan']

In [9]:
#测试集做预测,保持与features中的columns一致
test_fea = test1[features.columns]

test_fea['fea_hash'] = test_fea['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_fea['fea1_hash'] = test_fea['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

test_fea['version'] = test_fea['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
test_fea['lan'] = all_df[all_df['label'].isnull()]['lan']

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

def ensemble_model(clf,train_x,train_y,test):
    #采用十折交叉验证
    sk = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 2021)
    prob = [] #记录最终结果
    mean_acc = 0 #记录平均准确率
    for k,(train_index,val_index) in enumerate(sk.split(train_x,train_y)):
        train_x_real = train_x.iloc[train_index]
        train_y_real = train_y.iloc[train_index]
        val_x = train_x.iloc[val_index]
        val_y = train_y.iloc[val_index]
        # 子模型训练
        clf = clf.fit(train_x_real,train_y_real)
        val_y_pred = clf.predict(val_x)
        #子模型评估
        acc_val = accuracy_score(val_y,val_y_pred)
        print('第{}个子模型 acc{}'.format(k+1,acc_val))
        mean_acc += acc_val/5
        #子模型预测0,1
        test_y_pred = clf.predict_proba(test)[:,-1] #soft得到概率值
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob = sum(prob)/5
    return mean_prob

In [11]:
import xgboost as xgb
# 使用XgBoost训练
clf = xgb.XGBClassifier(
            max_depth=15, learning_rate=0.05, n_estimators=2000, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, 
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5,n_jobs=-1
)
result = ensemble_model(clf,features,labels,test_fea)
result

[16:43:21] WARNING: ../src/learner.cc:541: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [12]:
#保存结果
a = pd.DataFrame(test1['sid'])
a['label'] =  result
b = a
#转换为二分类
a['label'] = a['label'].apply(lambda x:0 if x<0.5 else 1)
a.to_csv('version7_XGB2000.csv',index = False)
b.to_csv('version7_XGB_result.csv',index = False)

In [13]:
a

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1


In [14]:
print(train)

        Unnamed: 0  android_id  apptype  carrier  dev_height  dev_ppi  \
0                0      316361     1199  46000.0         0.0      0.0   
1                1      135939      893      0.0         0.0      0.0   
2                2      399254      821      0.0       760.0      0.0   
3                3       68983     1004  46000.0      2214.0      0.0   
4                4      288999     1076  46000.0      2280.0      0.0   
...            ...         ...      ...      ...         ...      ...   
499995      499995      392477     1028  46000.0      1920.0      3.0   
499996      499996      346134     1001      0.0      1424.0      0.0   
499997      499997      499635      761  46000.0      1280.0      0.0   
499998      499998      239786      917  46001.0       960.0      0.0   
499999      499999      270531      929  46000.0      2040.0      3.0   

        dev_width  label    lan  media_id  ...       os    osv package  \
0             0.0      1    NaN       104  ...  a